# Data Collection
## !!!!!!!!After running the notebook, delete the county_level folder inside data\data_raw\BEA_Industry_Factors!!!!!!!!!
Getting data from 2012 to 2016  
Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
Years Prior to 2012 also only have the totals for all sectors not each sector.  
Years after 2017 not available  
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html  
ended up not using 2017 since the NAICS 2017 wasn't available in master_crrosswalk

In [30]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [31]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'

# Create new folders if needed
#state_data_dir = out_data_dir / 'state_level'
#if not state_data_dir.exists():
#    state_data_dir.mkdir(parents=True)
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)
    
#zip_data_dir = out_data_dir / 'zip_level'
#if not county_data_dir.exists():
#    county_data_dir.mkdir(parents=True)

In [32]:
county_data_dir

WindowsPath('C:/Users/Owner/Documents/modelearth/community/info/data/data_raw/BEA_Industry_Factors/county_level')

In [33]:
# Load the state FIPS codes key
state_fips = pd.read_csv(out_data_dir / 'state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [34]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_county_cbp(fips, state, years):
    count=0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year>=2000 and year<=2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2003 and year<=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2008 and year<=2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year>=2012 and year<=2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year==2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [152]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     #years=range(2012,2018)
     years=range(2012,2017)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2008
  > Finished CSV for year2008
Getting data for state: Alabama	year: 2009
  > Finished CSV for year2009
Getting data for state: Alabama	year: 2010
  > Finished CSV for year2010
Getting data for state: Alabama	year: 2011
  > Finished CSV for year2011
Getting data for state: Alaska	year: 2008
  > Finished CSV for year2008
Getting data for state: Alaska	year: 2009
  > Finished CSV for year2009
Getting data for state: Alaska	year: 2010
  > Finished CSV for year2010
Getting data for state: Alaska	year: 2011
  > Finished CSV for year2011
Getting data for state: Arizona	year: 2008
  > Finished CSV for year2008
Getting data for state: Arizona	year: 2009
  > Finished CSV for year2009
Getting data for state: Arizona	year: 2010
  > Finished CSV for year2010
Getting data for state: Arizona	year: 2011
  > Finished CSV for year2011
Getting data for state: Arkansas	year: 2008
  > Finished CSV for year2008
Getting data for state: Arkansas	year: 2009
  > Finish

KeyboardInterrupt: 

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS2012 files that we downloaded

In [133]:
# Load the 2012 data

def load_all_states2012(bea_data_dir):
    
    for i in range(2012,2017):
        #print(i)
        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:
            #print(f)
            
            df = pd.read_csv(f,encoding='latin-1',dtype={"NAICS2012": str})
            if 'Unnamed: 11' in df.columns:
                df=df.drop("Unnamed: 11", axis=1)
            if 'Unnamed: 10' in df.columns:
                df=df.drop("Unnamed: 10", axis=1)
            #df = pd.read_csv(f)
            #df[df['NAICS2012'].str.len() != 5]
            #b=df[df['NAICS2012'].str.len() == 5]
            df['is5'] = df['NAICS2012'].apply(lambda x: 'True' if len(x) == 5 else 'False')
                #print("hh"+df.NAICS2012)
                #print("hghg"+len(df.NAICS2012))
            #df['NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2] if df['is5']=='False' else 1).apply(int)
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df['NAICS2012']
            df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
                #print("hgh"+df.NAICS2012[1:2])
                #if (df.NAICS2012.apply(lambda v: v[0:1]).apply(int)==1).bool():
                #    print("bye")

            yield df
    
df = pd.concat(load_all_states2012(county_data_dir)).drop("is5", axis=1)

# test = list(load_all_states(bea_data_dir))

# df

In [9]:
df=df.drop("county", axis=1)

### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [10]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])
# # Remove the state_level data
# df_state = df[df['fips'].str.len() == 2]
# df_county = df[df['fips'].str.len() != 2]

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

In [11]:
#df.to_csv("allll.csv")

### Group all data per sector for  every year and average them  
Does it make sense to sum up instead of average?!

In [12]:
newDF=df.groupby(['fips','state','COUNTY','GEO_TTL','NAICS2012','NAICS2012_TTL','NAICS_Sector'],as_index=False).mean()
newDF=newDF.drop("YEAR", axis=1)

In [13]:
#newDF=newDF.astype({'fips': 'int'})
newDF=newDF.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [103]:
newDF.tail(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
2365201,56045,56,45,"Weston County, Wyoming",71394,Fitness and recreational sports centers,71,1.00,0.0,0.0
2365202,56045,56,45,"Weston County, Wyoming",713940,Fitness and recreational sports centers,71,1.00,0.0,0.0
2365203,56045,56,45,"Weston County, Wyoming",72,Accommodation and food services,72,25.00,197.6,2304.2
2365204,56045,56,45,"Weston County, Wyoming",721,Accommodation,72,6.40,24.4,489.8
2365205,56045,56,45,"Weston County, Wyoming",7211,Traveler accommodation,72,6.40,24.4,489.8
2365206,56045,56,45,"Weston County, Wyoming",72111,Hotels (except casino hotels) and motels,72,6.40,24.4,489.8
2365207,56045,56,45,"Weston County, Wyoming",721110,Hotels (except casino hotels) and motels,72,6.40,24.4,489.8
2365208,56045,56,45,"Weston County, Wyoming",722,Food services and drinking places,72,18.60,132.2,1437.8
2365209,56045,56,45,"Weston County, Wyoming",7224,Drinking places (alcoholic beverages),72,2.60,1.6,12.0
2365210,56045,56,45,"Weston County, Wyoming",72241,Drinking places (alcoholic beverages),72,2.60,1.6,12.0


### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [15]:
def naics_level(df, naics_level):
    return df[df['relevant_naics'].str.len() == naics_level]

In [115]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2 = naics_level(newDF, 2).reset_index(drop=True)
df_naics_3 = naics_level(newDF, 3).reset_index(drop=True)
df_naics_4 = naics_level(newDF, 4).reset_index(drop=True)
df_naics_5 = naics_level(newDF, 5).reset_index(drop=True)
df_naics_6 = naics_level(newDF, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2 = df_naics_2[df_naics_2.relevant_naics != '00']
df_naics_3 = df_naics_3[df_naics_3.relevant_naics != '00']
df_naics_4 = df_naics_4[df_naics_4.relevant_naics != '00']
df_naics_5 = df_naics_5[df_naics_5.relevant_naics != '00']
df_naics_6 = df_naics_6[df_naics_6.relevant_naics != '00']

In [17]:
#s2=state_level(df_naics_2)
c2=county_level(df_naics_2)
#s3=state_level(df_naics_3)
#c3=county_level(df_naics_3)
#s4=state_level(df_naics_4)
c4=county_level(df_naics_4)
#s5=state_level(df_naics_5)
#c5=county_level(df_naics_5)
s6=state_level(df_naics_6)
#c6=county_level(df_naics_6)

In [18]:
#s2.to_csv('s2.tsv',sep='\t')
#c2.to_csv('c2.tsv',sep='\t')
#s3.to_csv('s3.tsv',sep='\t')
#c3.to_csv('c3.tsv',sep='\t')
#s4.to_csv('s4.tsv',sep='\t')
#c4.to_csv('c4.tsv',sep='\t')
#s5.to_csv('s5.tsv',sep='\t')
#c5.to_csv('c5.tsv',sep='\t')
#s6.to_csv('s6.tsv',sep='\t')
#c6.to_csv('c6.tsv',sep='\t')

In [19]:
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,999,Alabama -- Statewide,00,Total for all sectors,00,405.80,42373.4,1333508.6
1,01,1,999,Alabama -- Statewide,21,"Mining, quarrying, and oil and gas extraction",21,18.00,386.4,27332.2
2,01,1,999,Alabama -- Statewide,211,Oil and gas extraction,21,4.20,31.6,3418.2
3,01,1,999,Alabama -- Statewide,2111,Oil and gas extraction,21,4.20,31.6,3418.2
4,01,1,999,Alabama -- Statewide,21111,Oil and gas extraction,21,4.20,31.6,3418.2
...,...,...,...,...,...,...,...,...,...,...
2365246,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,1.00,0.0,0.0
2365247,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.00,0.0,0.0
2365248,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,1.00,0.0,0.0
2365249,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,1.00,0.0,0.0


In [90]:
#newDF.to_csv('all.tsv',sep='\t')

# The statewide data does not include NAICS starting with 1!

In [20]:
a=county_level(df_naics_2)

In [21]:
county_level(df_naics_6)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
156,01001,1,1,"Autauga County, Alabama",113310,Logging,11,6.0,54.8,2611.2
157,01001,1,1,"Autauga County, Alabama",115112,"Soil preparation, planting, and cultivating",11,1.0,0.0,0.0
158,01001,1,1,"Autauga County, Alabama",115310,Support activities for forestry,11,1.0,0.0,0.0
159,01001,1,1,"Autauga County, Alabama",212319,Other crushed and broken stone mining and quar...,21,1.0,0.0,0.0
160,01001,1,1,"Autauga County, Alabama",212321,Construction sand and gravel mining,21,1.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
892472,56045,56,45,"Weston County, Wyoming",813110,Religious organizations,81,8.6,10.6,226.8
892473,56045,56,45,"Weston County, Wyoming",813312,"Environment, conservation and wildlife organiz...",81,1.0,0.0,0.0
892474,56045,56,45,"Weston County, Wyoming",813319,Other social advocacy organizations,81,1.0,0.0,0.0
892475,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,1.0,0.0,0.0


In [28]:
a=df_naics_6[df_naics_6.id=='13001'].sort_values(by=['payann'],ascending=False).head(50)
#a.to_csv("investigate.csv")

# NAICS code to name translation

In [29]:
NAICS_codes = pd.read_csv(out_data_dir / 'Crosswalk_MasterCrosswalk.csv', usecols=['2012_NAICS_Code', '2012_NAICS_Industry'])

In [30]:
NAICS_codes=NAICS_codes.rename(columns={"2012_NAICS_Code": "relevant_naics", "2012_NAICS_Industry": "industry_detail"})

In [31]:
NAICS_codes=NAICS_codes.dropna()

In [32]:
NAICS_codes=NAICS_codes.drop_duplicates()

In [33]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
13,111.0,Crop Production
19,1111.0,Oilseed and Grain Farming
21,11111.0,Soybean Farming
22,111110.0,Soybean Farming
...,...,...
3885,9281.0,National Security and International Affairs
3887,92811.0,National Security
3888,928110.0,National Security
3889,92812.0,International Affairs


In [34]:
#adding the row for Industries not classified
NAICS_codes
new_row = {'relevant_naics':99, 'industry_detail':"Industries not classified"}
#append row to the dataframe
NAICS_codes = NAICS_codes.append(new_row, ignore_index=True)

In [35]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
1,111.0,Crop Production
2,1111.0,Oilseed and Grain Farming
3,11111.0,Soybean Farming
4,111110.0,Soybean Farming
...,...,...
2210,92811.0,National Security
2211,928110.0,National Security
2212,92812.0,International Affairs
2213,928120.0,International Affairs


In [26]:
NAICS_codes.to_csv('industry_ID_list.csv')

# Making a states json

In [27]:
stateFips = pd.read_csv(out_data_dir / 'state_fips.csv')

In [37]:
stateFips=stateFips.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1)

In [38]:
#stateFips=stateFips.set_index('Name')

In [39]:
stateFips

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [31]:
stateFips.to_json('states.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

# Making county to fips csv

In [40]:
countyDF=c2[['GEO_TTL','id']].drop_duplicates()

In [41]:
countyDF

,GEO_TTL,id
16,"Autauga County, Alabama",01001
34,"Baldwin County, Alabama",01003
52,"Barbour County, Alabama",01005
70,"Bibb County, Alabama",01007
88,"Blount County, Alabama",01009
...,...,...
53916,"Sweetwater County, Wyoming",56037
53934,"Teton County, Wyoming",56039
53952,"Uinta County, Wyoming",56041
53969,"Washakie County, Wyoming",56043


In [42]:
countyDF['hascomma'] =countyDF['GEO_TTL'].apply(lambda x: 'True' if ',' in x else 'False')
countyDF
#countyDF['county'] = df.GEO_TTL.apply(lambda GTT: GTT.split(',')[0])

,GEO_TTL,id,hascomma
16,"Autauga County, Alabama",01001,True
34,"Baldwin County, Alabama",01003,True
52,"Barbour County, Alabama",01005,True
70,"Bibb County, Alabama",01007,True
88,"Blount County, Alabama",01009,True
...,...,...,...
53916,"Sweetwater County, Wyoming",56037,True
53934,"Teton County, Wyoming",56039,True
53952,"Uinta County, Wyoming",56041,True
53969,"Washakie County, Wyoming",56043,True


In [43]:
countyDF.loc[(countyDF['hascomma'] == 'True'), 'county'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[0])
countyDF.loc[(countyDF['hascomma'] == 'True'), 'state'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[1])

In [44]:
countyDF=countyDF[['state','county','id']].drop_duplicates()

In [45]:
countyDF

,state,county,id
16,Alabama,Autauga County,01001
34,Alabama,Baldwin County,01003
52,Alabama,Barbour County,01005
70,Alabama,Bibb County,01007
88,Alabama,Blount County,01009
...,...,...,...
53916,Wyoming,Sweetwater County,56037
53934,Wyoming,Teton County,56039
53952,Wyoming,Uinta County,56041
53969,Wyoming,Washakie County,56043


In [46]:
stats=stateFips.rename(columns={"Name": "state"})

In [47]:
stats=stats.drop("FIPS",axis=1)
stats

,state,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [48]:
countyDF=countyDF.merge(stats, on='state', how='left')

In [49]:
countyDF=countyDF.rename(columns={"Postal Code": "abvr"})
countyDF

,state,county,id,abvr
0,Alabama,Autauga County,01001,AL
1,Alabama,Baldwin County,01003,AL
2,Alabama,Barbour County,01005,AL
3,Alabama,Bibb County,01007,AL
4,Alabama,Blount County,01009,AL
...,...,...,...,...
3136,Wyoming,Sweetwater County,56037,WY
3137,Wyoming,Teton County,56039,WY
3138,Wyoming,Uinta County,56041,WY
3139,Wyoming,Washakie County,56043,WY


In [44]:
countyDF.to_csv('county_ID_list.csv')

# Saving the County level data

In [55]:
states=newDF.state.unique()
#states=[13]
df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
a=county_level(df_naics_2)
b=county_level(df_naics_4)
c=county_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2.tsv",sep='\t')
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6.tsv",sep='\t')
    #print(c1.dtypes)

GA
id                 object
state               int64
COUNTY              int64
GEO_TTL            object
relevant_naics     string
NAICS2012_TTL      object
NAICS_Sector       object
estab             float64
emp               float64
payann            float64
dtype: object


# saving the statewide data

In [51]:
states=newDF.state.unique()
#states=[13]

b1=county_level(df_naics_2)
c1=b1[b1.state==13]
c1.astype({'NAICS_Sector': 'int'})
d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d1.insert(0, 'id', 13)
d1.insert(1, 'COUNTY', 999)
d1.insert(2, 'GEO_TTL', 'Statewide')

b2=county_level(df_naics_4)
c2=b2[b2.state==13]
c2.astype({'NAICS_Sector': 'int'})
d2=c2.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d2.insert(0, 'id', 13)
d2.insert(1, 'COUNTY', 999)
d2.insert(2, 'GEO_TTL', 'Statewide')

b3=county_level(df_naics_6)
c3=b3[b3.state==13]
c3.astype({'NAICS_Sector': 'int'})
d3=c3.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d3.insert(0, 'id', 13)
d3.insert(1, 'COUNTY', 999)
d3.insert(2, 'GEO_TTL', 'Statewide')

for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a11=d1[d1.state==state]
    a11.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state.tsv",sep='\t')
    #print(a11)
    #print(a11.dtypes)
    
    b11=d2[d2.state==state]
    b11.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state.tsv",sep='\t')
    
    c11=d3[d3.state==state]
    c11.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state.tsv",sep='\t')

GA


### statewide from API, the wrong version!

In [46]:
"""
states=newDF.state.unique()
#states=[13]
a=state_level(df_naics_2)
b=state_level(df_naics_4)
c=state_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state_api.tsv",sep='\t')
    #print(a1)
    #print(a1.dtypes)
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state_api.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state_api.tsv",sep='\t')
"""

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


### statewide data from API, The correct version

In [123]:
state_data_dir = out_data_dir / 'state_level'
if not state_data_dir.exists():
    state_data_dir.mkdir(parents=True)

In [124]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_state_cbp(fips, state, years):
    count=0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year>=2000 and year<=2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2003 and year<=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2008 and year<=2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2012 and year<=2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year==2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(state_data_dir / f"industriesPerState_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [125]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     #years=range(2012,2018)
     years=range(2012,2017)
     get_state_cbp(fips, state, years)

Getting data for state: Alabama	year: 2012
  > Finished CSV for year2012
Getting data for state: Alabama	year: 2013
  > Finished CSV for year2013
Getting data for state: Alabama	year: 2014
  > Finished CSV for year2014
Getting data for state: Alabama	year: 2015
  > Finished CSV for year2015
Getting data for state: Alabama	year: 2016
  > Finished CSV for year2016
Getting data for state: Alaska	year: 2012
  > Finished CSV for year2012
Getting data for state: Alaska	year: 2013
  > Finished CSV for year2013
Getting data for state: Alaska	year: 2014
  > Finished CSV for year2014
Getting data for state: Alaska	year: 2015
  > Finished CSV for year2015
Getting data for state: Alaska	year: 2016
  > Finished CSV for year2016
Getting data for state: Arizona	year: 2012
  > Finished CSV for year2012
Getting data for state: Arizona	year: 2013
  > Finished CSV for year2013
Getting data for state: Arizona	year: 2014
  > Finished CSV for year2014
Getting data for state: Arizona	year: 2015
  > Finished 

  > Finished CSV for year2016
Getting data for state: Vermont	year: 2012
  > Finished CSV for year2012
Getting data for state: Vermont	year: 2013
  > Finished CSV for year2013
Getting data for state: Vermont	year: 2014
  > Finished CSV for year2014
Getting data for state: Vermont	year: 2015
  > Finished CSV for year2015
Getting data for state: Vermont	year: 2016
  > Finished CSV for year2016
Getting data for state: Virginia	year: 2012
  > Finished CSV for year2012
Getting data for state: Virginia	year: 2013
  > Finished CSV for year2013
Getting data for state: Virginia	year: 2014
  > Finished CSV for year2014
Getting data for state: Virginia	year: 2015
  > Finished CSV for year2015
Getting data for state: Virginia	year: 2016
  > Finished CSV for year2016
Getting data for state: Washington	year: 2012
  > Finished CSV for year2012
Getting data for state: Washington	year: 2013
  > Finished CSV for year2013
Getting data for state: Washington	year: 2014
  > Finished CSV for year2014
Getting

In [134]:
df_state = pd.concat(load_all_states2012(state_data_dir)).drop("is5", axis=1)

In [135]:
df_state

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,NAICS_Sector
0,0400000US01,Alabama,0,2012,00,Total for all sectors,97938,1585761,61284931,1,00
1,0400000US01,Alabama,0,2012,11,"Agriculture, forestry, fishing and hunting",785,6028,212695,1,11
2,0400000US01,Alabama,0,2012,238140,Masonry contractors,174,1548,50764,1,23
3,0400000US01,Alabama,0,2012,113,Forestry and logging,576,4184,151238,1,11
4,0400000US01,Alabama,0,2012,1131,Timber tract operations,37,216,9471,1,11
...,...,...,...,...,...,...,...,...,...,...,...
1613,0400000US56,Wyoming,0,2016,813930,Labor unions and similar labor organizations,49,252,4912,56,81
1614,0400000US56,Wyoming,0,2016,81394,Political organizations,7,13,508,56,81
1615,0400000US56,Wyoming,0,2016,813940,Political organizations,7,13,508,56,81
1616,0400000US56,Wyoming,0,2016,81399,"Other similar organizations (except business, ...",19,67,2776,56,81


In [137]:
df_state['fips'] = df_state.GEO_ID.apply(lambda GID: GID.split('US')[1])
df_state

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,NAICS_Sector,fips
0,0400000US01,Alabama,0,2012,00,Total for all sectors,97938,1585761,61284931,1,00,01
1,0400000US01,Alabama,0,2012,11,"Agriculture, forestry, fishing and hunting",785,6028,212695,1,11,01
2,0400000US01,Alabama,0,2012,238140,Masonry contractors,174,1548,50764,1,23,01
3,0400000US01,Alabama,0,2012,113,Forestry and logging,576,4184,151238,1,11,01
4,0400000US01,Alabama,0,2012,1131,Timber tract operations,37,216,9471,1,11,01
...,...,...,...,...,...,...,...,...,...,...,...,...
1613,0400000US56,Wyoming,0,2016,813930,Labor unions and similar labor organizations,49,252,4912,56,81,56
1614,0400000US56,Wyoming,0,2016,81394,Political organizations,7,13,508,56,81,56
1615,0400000US56,Wyoming,0,2016,813940,Political organizations,7,13,508,56,81,56
1616,0400000US56,Wyoming,0,2016,81399,"Other similar organizations (except business, ...",19,67,2776,56,81,56


In [138]:
newDF_state=df_state.groupby(['fips','state','COUNTY','GEO_TTL','NAICS2012','NAICS2012_TTL','NAICS_Sector'],as_index=False).mean()
newDF_state=newDF_state.drop("YEAR", axis=1)

In [139]:
#newDF=newDF.astype({'fips': 'int'})
newDF_state=newDF_state.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF_state

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
0,01,1,0,Alabama,00,Total for all sectors,00,98270.8,1620103.4,64865139.2
1,01,1,0,Alabama,11,"Agriculture, forestry, fishing and hunting",11,797.4,6396.6,240655.6
2,01,1,0,Alabama,113,Forestry and logging,11,580.8,4342.8,169322.4
3,01,1,0,Alabama,1131,Timber tract operations,11,44.2,290.2,14340.4
4,01,1,0,Alabama,11311,Timber tract operations,11,44.2,290.2,14340.4
...,...,...,...,...,...,...,...,...,...,...
95140,56,56,0,Wyoming,81394,Political organizations,81,7.6,14.0,673.8
95141,56,56,0,Wyoming,813940,Political organizations,81,7.6,14.0,673.8
95142,56,56,0,Wyoming,81399,"Other similar organizations (except business, ...",81,19.8,55.6,2692.8
95143,56,56,0,Wyoming,813990,"Other similar organizations (except business, ...",81,19.8,55.6,2692.8


In [140]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2_state = naics_level(newDF_state, 2).reset_index(drop=True)
df_naics_3_state = naics_level(newDF_state, 3).reset_index(drop=True)
df_naics_4_state = naics_level(newDF_state, 4).reset_index(drop=True)
df_naics_5_state = naics_level(newDF_state, 5).reset_index(drop=True)
df_naics_6_state = naics_level(newDF_state, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2_state = df_naics_2_state[df_naics_2_state.relevant_naics != '00']
df_naics_3_state = df_naics_3_state[df_naics_3_state.relevant_naics != '00']
df_naics_4_state = df_naics_4_state[df_naics_4_state.relevant_naics != '00']
df_naics_5_state = df_naics_5_state[df_naics_5_state.relevant_naics != '00']
df_naics_6_state = df_naics_6_state[df_naics_6_state.relevant_naics != '00']

In [141]:
df_naics_2_state

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
1,01,1,0,Alabama,11,"Agriculture, forestry, fishing and hunting",11,797.4,6396.6,240655.6
2,01,1,0,Alabama,21,"Mining, quarrying, and oil and gas extraction",21,237.4,7397.8,511247.2
3,01,1,0,Alabama,22,Utilities,22,414.6,13088.4,1150979.2
4,01,1,0,Alabama,23,Construction,23,7372.6,79101.8,3669112.4
5,01,1,0,Alabama,42,Wholesale trade,42,5356.6,71802.8,3862200.8
...,...,...,...,...,...,...,...,...,...,...
895,56,56,0,Wyoming,62,Health care and social assistance,62,1929.6,32270.8,1425619.4
896,56,56,0,Wyoming,71,"Arts, entertainment, and recreation",71,431.6,4435.6,92317.0
897,56,56,0,Wyoming,72,Accommodation and food services,72,1813.0,28312.6,538193.4
898,56,56,0,Wyoming,81,Other services (except public administration),81,1813.4,8551.2,276770.0


In [142]:

states=newDF.state.unique()
#states=[13]
a=df_naics_2_state
b=df_naics_4_state
c=df_naics_6_state
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state_api.tsv",sep='\t')
    #print(a1)
    #print(a1.dtypes)
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state_api.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state_api.tsv",sep='\t')


AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


# investigation!

In [90]:
bio_input = "113000,321113,113310,32121,32191,562213,322121,322110,"
bio_output = "325211,325991,3256,335991,325120,326190,"
green_energy = "221117,221111,221113,221114,221115,221116,221118,"
fossil_energy = "221112,324110"
cat_filter = []
cat_filter = (bio_input + bio_output + green_energy + fossil_energy).split(',')
print(cat_filter)
cat_filter2=[]
for i in cat_filter:
    #print( i[0:4])
    cat_filter2.append(i[0:4])
cat_filter2

['113000', '321113', '113310', '32121', '32191', '562213', '322121', '322110', '325211', '325991', '3256', '335991', '325120', '326190', '221117', '221111', '221113', '221114', '221115', '221116', '221118', '221112', '324110']


['1130',
 '3211',
 '1133',
 '3212',
 '3219',
 '5622',
 '3221',
 '3221',
 '3252',
 '3259',
 '3256',
 '3359',
 '3251',
 '3261',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '3241']

In [107]:
df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
dfp=df_naics_6[(df_naics_6.id=='13285') |  (df_naics_6.id=='13263') |  (df_naics_6.id=='13199') |  (df_naics_6.id=='13149') |  (df_naics_6.id=='13145') |  (df_naics_6.id=='13007')]
dfp=dfp[(dfp.relevant_naics.str.startswith('11')) | (dfp.relevant_naics.str.startswith('1130')) | (dfp.relevant_naics.str.startswith('3211')) |  (dfp.relevant_naics.str.startswith('1133')) | (dfp.relevant_naics.str.startswith('3212')) | (dfp.relevant_naics.str.startswith('3219')) | (dfp.relevant_naics.str.startswith('5622'))| (dfp.relevant_naics.str.startswith('3221'))| (dfp.relevant_naics.str.startswith('3252'))| (dfp.relevant_naics.str.startswith('3259'))| (dfp.relevant_naics.str.startswith('3256'))| (dfp.relevant_naics.str.startswith('3359'))| (dfp.relevant_naics.str.startswith('3251'))| (dfp.relevant_naics.str.startswith('3261'))| (dfp.relevant_naics.str.startswith('2211'))| (dfp.relevant_naics.str.startswith('3241'))]
dfp

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
132267,13007,13,7,"Baker County, Georgia",115114,Postharvest crop activities (except cotton gin...,11,1.0,0.0,0.0
152204,13145,13,145,"Harris County, Georgia",113210,Forest nurseries and gathering of forest products,11,1.0,0.0,0.0
152205,13145,13,145,"Harris County, Georgia",113310,Logging,11,2.4,1.4,54.6
152206,13145,13,145,"Harris County, Georgia",115210,Support activities for animal production,11,1.0,0.0,0.0
152207,13145,13,145,"Harris County, Georgia",115310,Support activities for forestry,11,2.2,0.0,0.0
152209,13145,13,145,"Harris County, Georgia",221112,Fossil fuel electric power generation,22,1.0,0.0,0.0
152210,13145,13,145,"Harris County, Georgia",221122,Electric power distribution,22,3.0,0.0,0.0
152239,13145,13,145,"Harris County, Georgia",321113,Sawmills,32,1.0,0.0,0.0
152240,13145,13,145,"Harris County, Georgia",321920,Wood container and pallet manufacturing,32,1.0,0.0,0.0
152241,13145,13,145,"Harris County, Georgia",321992,Prefabricated wood building manufacturing,32,1.0,0.0,0.0


In [108]:
dfp1=dfp.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['estab'],ascending=False).head(50)
dfp1

,NAICS_Sector,NAICS2012_TTL,state,relevant_naics,COUNTY,estab,emp,payann
1,11,Logging,13,113310,1041,14.4,48.6,2488.2
6,22,Electric power distribution,13,221122,1041,10.0,0.0,0.0
7,22,Fossil fuel electric power generation,13,221112,756,7.2,0.0,0.0
4,11,Support activities for forestry,13,115310,892,6.0,0.0,0.0
5,11,Timber tract operations,13,113110,896,4.6,0.0,0.0
9,32,All other plastics product manufacturing,13,326199,285,2.6,82.8,2653.2
23,32,Wood container and pallet manufacturing,13,321920,430,2.4,0.0,0.0
11,32,Asphalt paving mixture and block manufacturing,13,324121,430,2.2,0.0,0.0
14,32,Plastics material and resin manufacturing,13,325211,484,2.0,0.0,0.0
0,11,Forest nurseries and gathering of forest products,13,113210,344,2.0,0.0,0.0


In [116]:

df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
dfp=df_naics_6[(df_naics_6.id=='13001') |  (df_naics_6.id=='13005') |  (df_naics_6.id=='13127') |  (df_naics_6.id=='13161') |  (df_naics_6.id=='13229') |  (df_naics_6.id=='13305')]
dfp=dfp[(dfp.relevant_naics.str.startswith('11')) | (dfp.relevant_naics.str.startswith('1130')) | (dfp.relevant_naics.str.startswith('3211')) |  (dfp.relevant_naics.str.startswith('1133')) | (dfp.relevant_naics.str.startswith('3212')) | (dfp.relevant_naics.str.startswith('3219')) | (dfp.relevant_naics.str.startswith('5622'))| (dfp.relevant_naics.str.startswith('3221'))| (dfp.relevant_naics.str.startswith('3252'))| (dfp.relevant_naics.str.startswith('3259'))| (dfp.relevant_naics.str.startswith('3256'))| (dfp.relevant_naics.str.startswith('3359'))| (dfp.relevant_naics.str.startswith('3251'))| (dfp.relevant_naics.str.startswith('3261'))| (dfp.relevant_naics.str.startswith('2211'))| (dfp.relevant_naics.str.startswith('3241'))]
dfp

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
131874,13001,13,1,"Appling County, Georgia",113310,Logging,11,10.200000,34.6,1709.8
131875,13001,13,1,"Appling County, Georgia",115111,Cotton ginning,11,1.000000,0.0,0.0
131876,13001,13,1,"Appling County, Georgia",221113,Nuclear electric power generation,22,1.800000,0.0,0.0
131877,13001,13,1,"Appling County, Georgia",221122,Electric power distribution,22,5.200000,0.0,0.0
131894,13001,13,1,"Appling County, Georgia",321113,Sawmills,32,2.200000,0.0,0.0
131895,13001,13,1,"Appling County, Georgia",321911,Wood window and door manufacturing,32,1.000000,0.0,0.0
131896,13001,13,1,"Appling County, Georgia",321999,All other miscellaneous wood product manufactu...,32,1.000000,0.0,0.0
131897,13001,13,1,"Appling County, Georgia",325211,Plastics material and resin manufacturing,32,1.000000,0.0,0.0
131898,13001,13,1,"Appling County, Georgia",325620,Toilet preparation manufacturing,32,1.000000,0.0,0.0
132128,13005,13,5,"Bacon County, Georgia",113310,Logging,11,8.200000,40.6,1758.6


In [117]:
dfp1=dfp.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['payann'],ascending=False).head(50)
dfp1

,NAICS_Sector,NAICS2012_TTL,state,relevant_naics,COUNTY,estab,emp,payann
3,11,Logging,13,113310,828,55.000000,306.2,13025.6
4,11,Postharvest crop activities (except cotton gin...,13,115114,166,4.400000,32.2,1364.0
0,11,Cotton ginning,13,115111,162,2.000000,0.0,0.0
15,32,"Cut stock, resawing lumber, and planing",13,321912,161,1.000000,0.0,0.0
26,32,Wood window and door manufacturing,13,321911,1,1.000000,0.0,0.0
25,32,Wood preservation,13,321114,229,1.000000,0.0,0.0
24,32,Wood container and pallet manufacturing,13,321920,166,2.000000,0.0,0.0
23,32,Truss manufacturing,13,321214,127,1.000000,0.0,0.0
22,32,Toilet preparation manufacturing,13,325620,1,1.000000,0.0,0.0
21,32,Sawmills,13,321113,823,11.600000,0.0,0.0


## investigating the state total vs aggregation of counties

In [118]:
#states=newDF.state.unique()
states=[13]
a=state_level(df_naics_2)
b=state_level(df_naics_4)
c=state_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    
    b1=b[b.state==state]
    
    c1=c[c.state==state]

GA


In [120]:
a1.sort_values(by=['payann'],ascending=False).head(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS2012_TTL,NAICS_Sector,estab,emp,payann
6776,13,13,999,Georgia -- Statewide,56,Administrative and support and waste managemen...,56,136.0,85589.6,2519014.4
6774,13,13,999,Georgia -- Statewide,54,"Professional, scientific, and technical services",54,84.6,10120.2,494222.8
6772,13,13,999,Georgia -- Statewide,52,Finance and insurance,52,139.6,4974.4,410833.2
6770,13,13,999,Georgia -- Statewide,42,Wholesale trade,42,142.8,4117.2,379376.8
6771,13,13,999,Georgia -- Statewide,51,Information,51,46.6,1034.8,138206.4
6769,13,13,999,Georgia -- Statewide,23,Construction,23,7.8,1078.8,50349.8
6778,13,13,999,Georgia -- Statewide,62,Health care and social assistance,62,11.4,434.8,29836.4
6775,13,13,999,Georgia -- Statewide,55,Management of companies and enterprises,55,12.0,155.2,18746.2
6777,13,13,999,Georgia -- Statewide,61,Educational services,61,7.8,480.8,11528.2
6781,13,13,999,Georgia -- Statewide,81,Other services (except public administration),81,8.0,240.8,11088.8


In [121]:
states=[13]

b1=county_level(df_naics_2)
c1=b1[b1.state==13]
c1.astype({'NAICS_Sector': 'int'})
d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d1.insert(0, 'id', 13)
d1.insert(1, 'COUNTY', 999)
d1.insert(2, 'GEO_TTL', 'Statewide')

b2=county_level(df_naics_4)
c2=b2[b2.state==13]
c2.astype({'NAICS_Sector': 'int'})
d2=c2.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d2.insert(0, 'id', 13)
d2.insert(1, 'COUNTY', 999)
d2.insert(2, 'GEO_TTL', 'Statewide')

b3=county_level(df_naics_6)
c3=b3[b3.state==13]
c3.astype({'NAICS_Sector': 'int'})
d3=c3.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop("COUNTY",axis=1)
d3.insert(0, 'id', 13)
d3.insert(1, 'COUNTY', 999)
d3.insert(2, 'GEO_TTL', 'Statewide')

for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a11=d1[d1.state==state]

    
    b11=d2[d2.state==state]
    
    c11=d3[d3.state==state]


GA


In [122]:
a11.sort_values(by=['payann'],ascending=False).head(50)

,id,COUNTY,GEO_TTL,NAICS_Sector,NAICS2012_TTL,state,relevant_naics,estab,emp,payann
12,13,999,Statewide,62,Health care and social assistance,13,62,23705.400000,465813.800000,2.150282e+07
8,13,999,Statewide,54,"Professional, scientific, and technical services",13,54,28647.550000,230140.200000,1.678608e+07
6,13,999,Statewide,52,Finance and insurance,13,52,14548.600000,154641.000000,1.254450e+07
4,13,999,Statewide,42,Wholesale trade,13,42,12916.466667,191408.400000,1.204379e+07
9,13,999,Statewide,55,Management of companies and enterprises,13,55,1754.600000,114608.900000,1.198956e+07
5,13,999,Statewide,51,Information,13,51,4070.050000,114195.200000,9.764440e+06
10,13,999,Statewide,56,Administrative and support and waste managemen...,13,56,11976.250000,259153.400000,9.337075e+06
3,13,999,Statewide,23,Construction,13,23,16551.650000,151712.400000,7.710719e+06
14,13,999,Statewide,72,Accommodation and food services,13,72,19476.200000,389064.000000,6.115754e+06
15,13,999,Statewide,81,Other services (except public administration),13,81,20825.600000,153870.400000,4.254155e+06


# Estimation for 0 values

In [41]:
#states=stateFips.FIPS.unique()
states=[13]
naics=[2,4,6]
#naics=[6]
for state in states:
    for i in naics: 
        stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
        print(stateName+str(i))
        everything=pd.read_csv(f"usa/{stateName}/industries_state{state}_naics{i}_state_api.tsv",sep='\t')
        #print("everything")
        #print(everything.shape)
        known=pd.read_csv(f"usa/{stateName}/industries_state{state}_naics{i}_state.tsv",sep='\t')
        countylevel=pd.read_csv(f"usa/{stateName}/industries_state{state}_naics{i}.tsv",sep='\t')
        #print("countylevel")
        #print(countylevel.shape)
        valueDif=everything.merge(known[['relevant_naics','estab','emp','payann']], on='relevant_naics', how='left')
        valueDif['payann_diff']=valueDif['payann_x']-valueDif['payann_y']
        valueDif['emp_diff']=valueDif['emp_x']-valueDif['emp_y']
        valueDif['estab_diff']=valueDif['estab_x']-valueDif['estab_y']
        payannemp0=countylevel[(countylevel['payann']==0) & (countylevel['estab']>0) & (countylevel['emp']==0)]
        #print("os")
        #print(payannemp0.shape)
        middle0=payannemp0.drop(['state','Unnamed: 0','GEO_TTL','NAICS_Sector','emp','payann','id','COUNTY'],axis=1)
        estab0s=middle0.groupby(['relevant_naics','NAICS2012_TTL'],as_index=False).sum()
        estab0s=estab0s.rename(columns={'estab':'total_estabs'})
        merged1=payannemp0.merge(estab0s[['relevant_naics','total_estabs']], on='relevant_naics', how='left')
        merged1['estab_ratio']=merged1['estab']/merged1['total_estabs']
        merged2=merged1.merge(valueDif[['relevant_naics','payann_diff','emp_diff']], on='relevant_naics', how='left').drop("Unnamed: 0",axis=1)
        merged2=merged2.drop(['payann','emp'],axis=1)
        merged2['payann']=merged2['payann_diff']*merged2['estab_ratio']
        #(state total minus the known county-totals) * (establishments in county / establishments for all counties without county values)
        #for establishments for all counties without county values:
        #first:find the county level data that have payann and emp of 0, and use their number of establishment
        merged2['emp']=merged2['emp_diff']*merged2['estab_ratio']
        merged2=merged2.drop(['total_estabs','estab_ratio','payann_diff','emp_diff'],axis=1)
        #print("merges2")
        #print(merged2.shape)
        merged3=payannemp0.merge(valueDif[['relevant_naics','payann_diff','emp_diff','estab_diff']], on='relevant_naics', how='left').drop("Unnamed: 0",axis=1)
        merged3=merged3.drop(['payann','emp'],axis=1)
        merged3['estab_scale']=merged3['estab']/merged3['estab_diff']
        merged3['payann']=merged3['payann_diff']*merged3['estab_scale']
        merged3['emp']=merged3['emp_diff']*merged3['estab_scale']
        merged3.loc[merged3['emp'] <0, 'emp'] = 0
        #second:find the number of establishments by taking the difference of state-level data from api, and the aggregation of county-level data.
        #this one gives 0s for situations where the difference in number of estabs is less than 0.
        merged3.loc[merged3['payann'] <0, 'payann'] = 0
        merged3=merged3.drop(['payann_diff','emp_diff','estab_diff','estab_scale'],axis=1)
        #third:state total * (establishments in county / establishments for entire state)
        payannemp0s=payannemp0.drop(['emp','payann'],axis=1)
        merged4=payannemp0s.merge(everything[['relevant_naics','estab','emp','payann']], on='relevant_naics', how='left')
        merged4['estab_scale']=merged4['estab_x']/merged4['estab_y']
        merged4['payann']=merged4['payann']*merged4['estab_scale']
        merged4['emp']=merged4['emp']*merged4['estab_scale']
        merged4=merged4.rename(columns={'estab_x':'estab'})
        merged4=merged4.drop(['estab_y','estab_scale','Unnamed: 0'],axis=1)

        payannemppos=countylevel[(countylevel['payann']>0) | (countylevel['emp']>0)]
        payannemppos=payannemppos.drop('Unnamed: 0',axis=1)
        #print("non os")
        #print(payannemppos.shape)
        merged2['estimate']='yes'
        
        
        est1=pd.concat([payannemppos,merged2])
        #print("final1")
        #print(est1.shape)
        est2=pd.concat([payannemppos,merged3])
        est3=pd.concat([payannemppos,merged4])
        est1.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est1.tsv",sep='\t')
        #est2.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est2.tsv",sep='\t')
        est3.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est3.tsv",sep='\t')
        
        merged3.loc[merged3['estimate'] !='yes', 'estimate'] = 'no'
        print(est1)

GA2


KeyError: 'estimate'